Airlines 延误数据集 BTTWD 实验

本 notebook 按步骤运行：环境准备 → 加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD 实验 → 桶级分析。


In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_holdout_experiment,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'airlines_delay.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')


【INFO】【2025-11-25 21:58:47】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\airlines_delay.yaml
【INFO】【2025-11-25 21:58:51】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')


【INFO】【2025-11-25 21:58:51】【配置-数据】数据集=airlines_delay_1m, k折=5, 目标列=DepDelay, 正类="1"
【INFO】【2025-11-25 21:58:51】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-25 21:58:51】【配置-基线】LogReg启用=False, RandomForest启用=False, KNN启用=False, XGBoost启用=True
【INFO】【2025-11-25 21:58:51】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（延误/不延误）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='延误 vs 未延误比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始 DepDelay 的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是 "DepDelay"
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Airlines 原始数据加载与基本统计完成。')


【INFO】【2025-11-25 21:58:52】【数据加载】文本表格 ../data/airline/airlines_train_regression_200000.csv 已读取，样本数=200000，列数=10
【INFO】【2025-11-25 21:58:52】【目标变换】已按阈值 15.0 生成二分类标签列 label，正类取 > 15.0
【INFO】【2025-11-25 21:58:52】【数据集信息】名称=airlines_delay_1m，样本数=200000，目标列=label，正类比例=15.60%


,DepDelay,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,Origin,Dest,Distance,label
0,-7.0,4.0,26.0,5.0,935.0,1050.0,MQ,MIA,JAX,335.0,0
1,-2.0,2.0,10.0,6.0,740.0,911.0,DL,MLB,ATL,443.0,0
2,40.0,10.0,30.0,4.0,820.0,930.0,MQ,DFW,LIT,304.0,1
3,0.0,1.0,8.0,2.0,1220.0,1355.0,WN,OAK,PDX,543.0,0
4,0.0,5.0,7.0,7.0,1750.0,1835.0,NW,LIT,MEM,130.0,0


用于建模的标签列: label
原始目标列: DepDelay
count    200000.000000
mean          8.192070
std          28.680684
min        -534.000000
25%          -3.000000
50%           0.000000
75%           7.000000
max        1438.000000
Name: DepDelay, dtype: float64
【INFO】【2025-11-25 21:58:52】【步骤2摘要】Airlines 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")


【INFO】【2025-11-25 21:58:52】【预处理】连续特征=6个，类别特征=3个
【INFO】【2025-11-25 21:58:52】【预处理】编码后维度=718
【INFO】【2025-11-25 21:58:52】【预处理】编码特征维度=718，样本数=200000
【INFO】【2025-11-25 21:58:52】【步骤3摘要】特征预处理完成：连续=6，类别=3，编码维度=718。


In [5]:
# 步骤4：构建桶树并检查划分
feature_df_for_bucket = df_raw.drop(columns=[cfg['DATA']['target_col']])
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=feature_df_for_bucket.columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(feature_df_for_bucket)
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(
    lambda s: (s == cfg['DATA']['positive_label']).mean()
).values

display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')


【INFO】【2025-11-25 21:58:53】【桶树】已为样本生成桶ID，共 12374 个组合


,bucket_id,count,pos_rate
0,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,845,0.125000
1,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,839,0.166667
2,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,836,0.000000
3,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,832,0.200000
4,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,828,0.032258


【INFO】【2025-11-25 22:07:18】【步骤4摘要】桶树划分完成，共有 12374 个叶子桶。


In [6]:
# 步骤5：运行基线模型实验占位
# 基线部分在 run_kfold_experiments 内统一调度（仅在 use_kfold=True 时执行）
log_info('【步骤5】基线模型将在交叉验证模式中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')


【INFO】【2025-11-25 22:07:18】【步骤5】基线模型将在交叉验证模式中一并运行。
【INFO】【2025-11-25 22:07:18】【步骤5摘要】基线模型性能将作为后续对比基准。


In [8]:
# 步骤6：运行 BTTWD 实验（k 折或单次留出）
use_kfold_raw = cfg.get('DATA', {}).get('use_kfold', False)
if isinstance(use_kfold_raw, str):
    use_kfold = use_kfold_raw.strip().lower() in ['true', '1', 'yes']
else:
    use_kfold = bool(use_kfold_raw)

if use_kfold:
    log_info('【步骤6】检测到 use_kfold=True，进入 k 折实验。')
    results = run_kfold_experiments(X, y, feature_df_for_bucket, cfg)
    summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
    display(summary_df)
    summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
    fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
    plt.savefig(fig_compare, bbox_inches='tight')
    plt.close()
    log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')
else:
    log_info('【步骤6】use_kfold=False，执行单次留出验证流程。')
    holdout_metrics = run_holdout_experiment(X, y, feature_df_for_bucket, cfg)
    display(pd.DataFrame(holdout_metrics))
    log_info('【步骤6摘要】单次留出验证完成，指标已列出。')


【INFO】【2025-11-25 22:13:04】【步骤6】检测到 use_kfold=True，进入 k 折实验。
【INFO】【2025-11-25 22:13:04】【基线-XGB】使用模型自定义阈值=0.150（per_model 模式）


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:13:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:13:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:13:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



【INFO】【2025-11-25 22:14:18】【基线-XGB】整体指标：AUC_mean=0.680, AUC_std=0.003, BAC_mean=0.630, BAC_std=0.002, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.344, F1_std=0.002, Kappa_mean=0.145, Kappa_std=0.002, MCC_mean=0.190, MCC_std=0.003, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.230, Precision_std=0.001, Recall_mean=0.682, Recall_std=0.008, Regret_mean=0.504, Regret_std=0.001
【INFO】【2025-11-25 22:14:18】【K折实验】正在执行第 1/5 折...
【INFO】【2025-11-25 22:14:21】【桶树】已为样本生成桶ID，共 8364 个组合


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:14:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-25 22:14:31】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=AA 子桶样本不足（最小子桶 n=25 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=CO 子桶样本不足（最小子桶 n=12 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=DL 子桶样本不足（最小子桶 n=71 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=MQ 子桶样本不足（最小子桶 n=2 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=NW 子桶样本不足（最小子桶 n=22 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=OO 子桶样本不足（最小子桶 n=1 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=OTHER 子桶样本不足（最小子桶 n=134 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=UA 子桶样本不足（最小子桶 n=38 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=US 子桶样本不足（最小子桶 n=6 < 500），不再细分
[INFO][BT][2025-11-25 22:14:36] 桶 L1_UniqueCarrier=WN 子桶样本不足（最小子桶 n=1 < 500），不再细分
【INFO】【2025-11-25 22:14:36】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-25 22:14:41】【BTTWD】共生成

...

【INFO】【2025-11-25 22:15:36】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=AA 子桶样本不足（最小子桶 n=20 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=CO 子桶样本不足（最小子桶 n=12 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=DL 子桶样本不足（最小子桶 n=71 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=MQ 子桶样本不足（最小子桶 n=1 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=NW 子桶样本不足（最小子桶 n=23 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=OO 子桶样本不足（最小子桶 n=1 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=OTHER 子桶样本不足（最小子桶 n=141 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=UA 子桶样本不足（最小子桶 n=33 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=US 子桶样本不足（最小子桶 n=6 < 500），不再细分
[INFO][BT][2025-11-25 22:15:40] 桶 L1_UniqueCarrier=WN 子桶样本不足（最小子桶 n=1 < 500），不再细分
【INFO】【2025-11-25 22:15:40】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-25 22:15:45】【BTTWD】共生成

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:15:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-25 22:16:01】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=AA 子桶样本不足（最小子桶 n=25 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=CO 子桶样本不足（最小子桶 n=13 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=DL 子桶样本不足（最小子桶 n=74 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=MQ 子桶样本不足（最小子桶 n=3 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=NW 子桶样本不足（最小子桶 n=20 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=OO 子桶样本不足（最小子桶 n=1 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=OTHER 子桶样本不足（最小子桶 n=125 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=UA 子桶样本不足（最小子桶 n=37 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=US 子桶样本不足（最小子桶 n=6 < 500），不再细分
[INFO][BT][2025-11-25 22:16:05] 桶 L1_UniqueCarrier=WN 子桶样本不足（最小子桶 n=15 < 500），不再细分
【INFO】【2025-11-25 22:16:06】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-25 22:16:10】【BTTWD】共生

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.312773,0.006773,0.273689,0.007568,0.291921,0.007119,0.581283,0.003941,0.677833,...,0.171808,0.007903,0.171282,0.007921,0.038835,0.00907,0.035405,0.008622,0.457925,0.003080
1,XGBoost,0.230412,0.000689,0.681530,0.007705,0.344387,0.001665,0.630429,0.002182,0.679521,...,0.189871,0.003061,0.145075,0.001663,0.000000,0.00000,NaN,NaN,0.504080,0.001065


【INFO】【2025-11-25 22:16:12】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [9]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标（如存在）已整理，可用于局部化分析。')


,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,threshold_n_samples,n_all,pos_rate_all,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,L1_UniqueCarrier=OTHER,L1,NaN,26656,6554,0.165891,0.165853,0.30,0.25,0.481309,...,6554,33210,0.165884,NaN,NaN,NaN,NaN,NaN,1,0.165884
1,L1_UniqueCarrier=WN,L1,NaN,17926,4422,0.176727,0.181366,0.30,0.25,0.477838,...,4422,22348,0.177645,NaN,NaN,NaN,NaN,NaN,1,0.177645
2,L1_UniqueCarrier=DL,L1,NaN,16442,4153,0.133560,0.135324,0.35,0.30,0.401999,...,4153,20595,0.133916,NaN,NaN,NaN,NaN,NaN,1,0.133916
3,L1_UniqueCarrier=AA,L1,NaN,14665,3648,0.144357,0.138158,0.40,0.35,0.411732,...,3648,18313,0.143122,NaN,NaN,NaN,NaN,NaN,1,0.143122
4,L1_UniqueCarrier=US,L1,NaN,13433,3378,0.142708,0.151569,0.50,0.45,0.454707,...,3378,16811,0.144489,NaN,NaN,NaN,NaN,NaN,1,0.144489


【INFO】【2025-11-25 22:16:13】【步骤7摘要】桶级指标（如存在）已整理，可用于局部化分析。


In [10]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
results_dir = os.path.join(root_path, cfg['OUTPUT']['results_dir'])
figs_dir = os.path.join(root_path, cfg['OUTPUT']['figs_dir'])
os.makedirs(results_dir, exist_ok=True)
os.makedirs(figs_dir, exist_ok=True)
print(os.listdir(results_dir))
print(os.listdir(figs_dir))
log_info('【全部步骤完成】Airlines 数据集的 BT-TWD 实验结束。')


【INFO】【2025-11-25 22:16:13】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'bucket_thresholds_per_fold.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv']
['bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-25 22:16:13】【全部步骤完成】Airlines 数据集的 BT-TWD 实验结束。
